In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time
from spotipy.oauth2 import SpotifyClientCredentials
#!pip install spotipy --upgrade

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')
song_attributes_df = pd.read_csv('../data/songAttributes_1999-2019.csv')

In [9]:
#spotify_qualities_df.info()
updated_song_attributes_df = song_attributes_df.drop(['Unnamed: 0', 'Popularity', 'Album'], axis=1)
updated_song_attributes_df = updated_song_attributes_df.rename(columns={'Name': 'song', 'Artist': 'artist'})
updated_song_attributes_df['artist'] = updated_song_attributes_df['artist'].str.lower()
updated_song_attributes_df['song'] = updated_song_attributes_df['song'].str.lower()
updated_song_attributes_df['artist'].value_counts()

artist
oakenfold         1797
willie nelson     1586
bing crosby       1434
cash cash         1406
elvis presley     1312
                  ... 
vitamin c           31
teairra mari        31
julianne hough      27
wham!               27
chris lake          21
Name: count, Length: 989, dtype: int64

In [5]:
spotify_qualities_df = spotify_qualities_df.drop(['Popularity', 'Album'], axis=1)
updated_spotify_qualities_df = spotify_qualities_df.rename(columns={'Artist': 'artist'})
updated_spotify_qualities_df['artist'] = spotify_qualities_df['artist'].str.lower()
updated_spotify_qualities_df['song'] = spotify_qualities_df['song'].str.lower()
updated_spotify_qualities_df.head(20)

KeyError: "['Popularity', 'Album'] not found in axis"

In [6]:
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['date'].dt.year
updated_charts_df = charts_df.drop(['date', 'last-week', 'rank'], axis=1)
updated_charts_df['artist'] = updated_charts_df['artist'].str.lower()
updated_charts_df['song'] = updated_charts_df['song'].str.lower()
updated_charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   song            330087 non-null  object
 1   artist          330087 non-null  object
 2   peak-rank       330087 non-null  int64 
 3   weeks-on-board  330087 non-null  int64 
 4   year            330087 non-null  int32 
dtypes: int32(1), int64(2), object(2)
memory usage: 11.3+ MB


In [14]:
featured_artists= ['featuring', '&', 'with', 'x']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

updated_charts_df['featured_artist'] = updated_charts_df['artist'].apply(extract_featured)
updated_charts_df['main_artist'] = updated_charts_df['artist'].apply(extract_main_artist)
updated_charts_df.tail(20)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
330067,beautiful delilah,chuck berry,81,1,1958,None,chuck berry
330068,blip blop,bill doggett,82,1,1958,None,bill doggett
330069,try the impossible,lee andrews and the hearts,83,1,1958,None,lee andrews and the hearts
330070,summertime blues,eddie cochran,84,1,1958,None,eddie cochran
330071,got a match?,the daddy-o's,85,1,1958,None,the daddy-o's
330072,to be loved,jackie wilson,86,1,1958,None,jackie wilson
330073,jealousy,kitty wells,87,1,1958,None,kitty wells
330074,just like in the movies,the upbeats,88,1,1958,None,the upbeats
330075,blue boy,jim reeves,89,1,1958,None,jim reeves
330076,stay,the ames brothers,90,1,1958,None,the ames brothers


In [16]:
range_updated_charts_df = updated_charts_df[
    (updated_charts_df['year'] >= 1999) &
    (updated_charts_df['year'] <= 2023)]
range_updated_charts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119300 entries, 0 to 119299
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   song             119300 non-null  object
 1   artist           119300 non-null  object
 2   peak-rank        119300 non-null  int64 
 3   weeks-on-board   119300 non-null  int64 
 4   year             119300 non-null  int32 
 5   featured_artist  36520 non-null   object
 6   main_artist      119300 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 6.8+ MB


In [7]:
chart_and_att_df = pd.merge(updated_charts_df, updated_song_attributes_df, on='song', how='inner')
grouped_chart_and_att_df = chart_and_att_df.groupby('song').first().reset_index()
grouped_chart_and_att_df.head(20)

,song,artist_x,peak-rank,weeks-on-board,year,Acousticness,artist_y,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Speechiness,Tempo,TimeSignature,Valence
0,#1,nelly,22,20,2002,0.13300,gerald levert,0.434,262400,0.664,False,0.000000,0.0523,-5.743,0,0.1030,172.129,5,0.469
1,#beautiful,mariah carey featuring miguel,15,16,2013,0.32200,mariah carey,0.676,199947,0.755,True,0.000000,0.2980,-5.333,1,0.0388,107.030,4,0.425
2,#thatpower,will.i.am featuring justin bieber,17,16,2013,0.00112,will.i.am,0.797,279507,0.608,False,0.000077,0.0748,-6.096,0,0.0583,128.000,4,0.403
3,'til i can make it on my own,tammy wynette,84,5,1976,0.61900,billy gilman,0.543,190867,0.316,False,0.000053,0.2280,-10.936,1,0.0270,71.948,4,0.231
4,'til my baby comes home,luther vandross,29,16,1985,0.05670,luther vandross,0.804,332227,0.714,False,0.000006,0.0253,-6.714,0,0.1830,139.663,4,0.802
5,'til summer comes around,keith urban,58,16,2010,0.59300,keith urban,0.570,331467,0.629,False,0.000136,0.7700,-7.608,0,0.0331,127.907,4,0.308
6,(1-2-3-4-5-6-7) count the days,inez & charlie foxx,76,5,1968,0.54800,patti labelle,0.555,162653,0.700,False,0.000007,0.6210,-5.723,1,0.0508,77.985,4,0.703
7,(all i have to do is) dream,the nitty gritty dirt band,66,6,1975,0.82100,nitty,0.574,228773,0.423,False,0.017500,0.1630,-13.301,0,0.0239,95.994,4,0.640
8,(everything i do) i do it for you,bryan adams,1,22,1991,0.43900,kenny g,0.504,303027,0.370,False,0.007560,0.0715,-9.879,1,0.0290,129.987,4,0.196
9,(for god's sake) give more power to the people,the chi-lites,26,11,1971,0.03010,joss stone,0.638,224307,0.920,False,0.000142,0.2350,-4.762,0,0.0743,101.113,4,0.693


In [8]:
chart_att_and_qual_df = pd.merge(grouped_chart_and_att_df, updated_spotify_qualities_df, on=['year', 'song'], how='inner')
chart_att_and_qual_df.info()

KeyError: 'year'

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['featuring', '&', 'with']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

grouped_chart_and_att_df['featured_artist'] = grouped_chart_and_att_df['artist_x'].apply(extract_featured)
grouped_chart_and_att_df['main_artist'] = grouped_chart_and_att_df['artist_x'].apply(extract_main_artist)
grouped_chart_and_att_df['main_artist'].value_counts()

In [ ]:
grouped_lyrics_df = lyrics_df.groupby('song').first().reset_index()
grouped_lyrics_df

In [ ]:
charts_df.info()

In [ ]:
charts_df['artist'].value_counts()

In [ ]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
spotify_qualities_df = spotify_qualities_df.rename(columns={'Time_Signature': 'time_signature', 'Track': 'song', 'Year': 'year', 'Danceability': 'danceability', 'Energy': 'energy', 'Key': 'key', 'Loudness': 'loudness', 'Mode': 'mode', 'Speechiness': 'speechiness', 'Liveness': 'liveness', 'Valence': 'valence', 'Tempo': 'tempo'}) 
spotify_qualities_df.info()

In [ ]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [ ]:
charts_df.info()

In [ ]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df.info()

In [ ]:
range_charts_and_lyrics_df = charts_and_lyrics_df[
    (charts_and_lyrics_df['year'] >= 1999) &
    (charts_and_lyrics_df['year'] <= 2023)]
no_repeats_range_charts_and_lyrics_df = range_charts_and_lyrics_df.groupby('song').first().reset_index()
no_repeats_range_charts_and_lyrics_df.head(10)

In [ ]:
#charts_and_lyrics_qualities_df = pd.merge(charts_and_lyrics_df, spotify_qualities_df, on=['year', 'song', 'time_signature', 'danceability', 'energy', 'loudness', 'speechiness', 'liveness', 'valence', 'tempo'], how='inner')
#pd.set_option('display.max_columns', None)
#charts_and_lyrics_qualities_df= charts_and_lyrics_qualities_df['song'].nunique()
#charts_and_lyrics_qualities_df.head(30)

In [ ]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
#artists

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

In [ ]:
spotify_qualities_df.dtypes

In [ ]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['time_signature'].value_counts()

In [ ]:
charts_and_lyrics_df[charts_and_lyrics_df['time_signature'] == 5]

In [ ]:
spotify_qualities_df['year'].max()
charts_and_lyrics_df['year'].max()

In [ ]:
charts_and_lyrics_df.info()

In [ ]:
updated_charts_and_lyrics_df = no_repeats_range_charts_and_lyrics_df.combine_first(spotify_qualities_df.set_index('song')).reset_index()
updated_charts_and_lyrics_df.head()